<a href="https://colab.research.google.com/github/NickyTan8899/tjy/blob/main/pretrain_100epochs_aliked_disk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

animal_clef_2025_path = kagglehub.competition_download('animal-clef-2025')
hathawaytan_balanced_accuracy_path = kagglehub.notebook_output_download('hathawaytan/balanced-accuracy')
hathawaytan_baseline_with_wildfusion_path = kagglehub.notebook_output_download('hathawaytan/baseline-with-wildfusion')

print('Data source import complete.')


In [ ]:
!pip install git+https://github.com/WildlifeDatasets/wildlife-datasets@develop
!pip install git+https://github.com/WildlifeDatasets/wildlife-tools

In [ ]:
import os
import numpy as np
import pandas as pd
import timm
import torchvision.transforms as T
from wildlife_datasets.datasets import AnimalCLEF2025
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.similarity.wildfusion import SimilarityPipeline, WildFusion
from wildlife_tools.similarity.pairwise.lightglue import MatchLightGlue
from wildlife_tools.similarity.pairwise.loftr import MatchLOFTR
from wildlife_tools.features.local import AlikedExtractor,SuperPointExtractor,SiftExtractor,DiskExtractor
from wildlife_tools.similarity.calibration import IsotonicCalibration,LogisticCalibration
import sys
# sys.path.append('/kaggle/input/balanced-accuracy')  # 添加路径
# from metric import score,BAKS,BAUS

def create_sample_submission(dataset_query, predictions, file_name='submission.csv'):
    df = pd.DataFrame({
        'image_id': dataset_query.metadata['image_id'],
        'identity': predictions
    })
    df.to_csv(file_name, index=False)

In [ ]:

root = animal_clef_2025_path
root

In [ ]:
# Loading the dataset
dataset = AnimalCLEF2025(root, load_label=True,transform=transform)
dataset_database = dataset.get_subset(dataset.metadata['split'] == 'database')
dataset_query = dataset.get_subset(dataset.metadata['split'] == 'query')
dataset_calibration = AnimalCLEF2025(root, df=dataset_database.metadata[:100], load_label=True)
meta=dataset.metadata.query('split == "database"')
num_classes = meta["identity"].nunique()
print(f"种类数: {num_classes}")
n_query=len(dataset_query)
n_query

In [ ]:
from itertools import chain
import torch
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD
import wildlife_tools
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.train import ArcFaceLoss, BasicTrainer
import importlib
import wildlife_tools.data.dataset
import gc

# 清理 Python 层的垃圾
gc.collect()

# 清除 CUDA 缓存（释放未使用的显存）
torch.cuda.empty_cache()
importlib.reload(wildlife_tools.data.dataset)

# 使用类
WildlifeDataset = wildlife_tools.data.dataset.WildlifeDataset
transform = T.Compose([
    T.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    T.RandAugment(num_ops=2, magnitude=20),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])
train_dataset=WildlifeDataset(root=root,metadata =meta,transform=transform ,load_label=True)
print(len(train_dataset))
# Backbone and loss configuration
backbone = timm.create_model('swin_base_patch4_window7_224', num_classes=0, pretrained=True)
with torch.no_grad():
    dummy_input = torch.randn(1, 3, 224, 224)
    embedding_size = backbone(dummy_input).shape[1]
    print(embedding_size)#1024
objective = ArcFaceLoss(num_classes=train_dataset.num_classes, embedding_size=embedding_size, margin=0.5, scale=64)


# Optimizer and scheduler configuration
params = chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)
min_lr = optimizer.defaults.get("lr") * 1e-3
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=min_lr)

importlib.reload(wildlife_tools.train.trainer)

# Step 2: 重新导入类（必须在 reload 之后！）
from wildlife_tools.train.trainer import BasicTrainer
# Setup training
trainer = BasicTrainer(
    dataset=train_dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    scheduler=scheduler,
    batch_size=64,
    accumulation_steps=2,
    num_workers=2,
    epochs=100,
    device='cuda',
)

trainer.train() # Call the modified train function
# trainer.save("/content/final_checkpoints", file_name="final_model.pth")

In [ ]:
# Loading the models

device = 'cuda'
model=backbone
pipelines = [


    SimilarityPipeline(
        matcher = MatchLightGlue(features='aliked'),
        extractor = AlikedExtractor(),
        transform = T.Compose([
            T.Resize([256, 256]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='disk'),
        extractor = DiskExtractor(),
        transform = T.Compose([
            T.Resize([256, 256]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),
]

matcher_mega = SimilarityPipeline(
    matcher = CosineSimilarity(),
    extractor = DeepFeatures(model=model, device=device, batch_size=16),
    transform = transform,
    calibration =IsotonicCalibration()
)

In [ ]:
# Calibrating the WildFusion

wildfusion = WildFusion(calibrated_pipelines = pipelines, priority_pipeline = matcher_mega)
wildfusion.fit_calibration(dataset_calibration, dataset_calibration)

In [ ]:
# Compute WildFusion similarity
similarity = wildfusion(dataset_query, dataset_database, B=25)

In [ ]:
pred_idx = similarity.argsort(axis=1)[:,-1]
pred_scores = similarity[range(n_query), pred_idx]
similarity.shape
labels = dataset_database.labels_string
pred_scores.shape

In [ ]:
for threshold in [0.2, 0.3, 0.4, 0.5, 0.6,0.7,0.8]:
    predictions = labels[pred_idx]
    predictions[pred_scores < threshold] = 'new_individual'
    create_sample_submission(dataset_query, predictions, file_name=f'sample_submission_{threshold}.csv')